#Parte 1 - Notebook de Processamento de Dados

##**Desafio**

Para aumentar o engajamento dos clientes, as empresas fazem uso de cupons que são enviados por meio de vários canais de marketing. Essa estratégia não apenas atrai os clientes para um serviço, mas também pode estabelecer um relacionamento de longo prazo, aumentando a probabilidade de transformar clientes ocasionais em consumidores recorrentes.

No entanto, ter uma estratégia eficiente de distribuição de ofertas é um desafio considerável devido aos múltiplos aspectos envolvidos:
- Diferentes tipos de ofertas disponíveis
- Diversos canais de marketing
- Variados perfis de consumo dos clientes
- Timing do envio das ofertas

##**Objetivo:** 
  1. Analise dos dados históricos de transações, ofertas e clientes
  2. Realizar o processamento, limpeza e preparação dos dados para modelagem
  3. Processo de treino e avaliação do modelo escolhido

## Dados fornecidos :

### offers.json
Contém os ids das ofertas e metadados de cada uma delas:
- id (string): id da oferta
- offer_type (string): o tipo da oferta (BOGO, discount, informational)
- min_value (int): valor mínimo para ativação da oferta
- duration (int): duração da oferta
- discount_value (int): valor do desconto
- channels (list of strings): canais de veiculação

### profile.json
Contém atributos de cerca de 17k clientes:
- age (int): idade do cliente na criação da conta
- registeredon (int): data de criação da conta
- gender (string): gênero do cliente
- id (string): id do cliente
- credit_card_limit (float): limite do cartão registrado

### transactions.json
Contém cerca de 300k eventos:
- event (str): descrição do evento (transação, oferta recebida, etc.)
- account_id (str): id do cliente
- time_since_test_start (int): tempo desde o começo do teste em dias (t=0)
- value (json): registra offer_id, desconto (reward) ou valor da transação


**Autor:** Natalia Ruivo

**Data:** Janeiro 2026




In [0]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import json



#1.0 Preparação do Ambiente e Ingestão de Dados

- Ambiente utilizado: Databricks Free Edition 

### 1.1 Download dos dados brutos
 - download do dataset do case compactado (.tar.gz) a partir de um bucket do Amazon S3, utilizando comandos de shell executados no ambiente do notebook.

### 1.2 Extração do dataset
- Conteúdo extraído disponibilizado no filesystem do ambiente para leitura.

### 1.3 Configuração de armazenamento governado (Unity Catalog)
- Foi criado um Volume no Unity Catalog, garantindo um local de armazenamento governado, persistente e com controle de acesso centralizado.

In [0]:
%sh
# Transferindo o arquivo ds-technical-evaluation-data.tar.gz para o diretório atual de execução
wget https://data-architect-test-source.s3.sa-east-1.amazonaws.com/ds-technical-evaluation-data.tar.gz

--2026-01-10 21:22:46--  https://data-architect-test-source.s3.sa-east-1.amazonaws.com/ds-technical-evaluation-data.tar.gz
Resolving data-architect-test-source.s3.sa-east-1.amazonaws.com (data-architect-test-source.s3.sa-east-1.amazonaws.com)... 3.5.234.194, 3.5.232.200, 3.5.232.222, ...
Connecting to data-architect-test-source.s3.sa-east-1.amazonaws.com (data-architect-test-source.s3.sa-east-1.amazonaws.com)|3.5.234.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7451952 (7.1M) [application/x-gzip]
Saving to: ‘ds-technical-evaluation-data.tar.gz.1’

     0K .......... .......... .......... .......... ..........  0%  560K 13s
    50K .......... .......... .......... .......... ..........  1%  258K 20s
   100K .......... .......... .......... .......... ..........  2%  335K 21s
   150K .......... .......... .......... .......... ..........  2%  498K 19s
   200K .......... .......... .......... .......... ..........  3% 30.5M 15s
   250K .......... .........

In [0]:
%sh
#descompactando o conteúdo e extraindo no diretório atual
tar -xvzf ds-technical-evaluation-data.tar.gz

._ds-technical-evaluation-data


tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'


ds-technical-evaluation-data/
ds-technical-evaluation-data/._offers.json


tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.metadata:kMDItemWhereFroms'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'


ds-technical-evaluation-data/offers.json
ds-technical-evaluation-data/._transactions.json


tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.metadata:kMDItemWhereFroms'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'


ds-technical-evaluation-data/transactions.json
ds-technical-evaluation-data/._profile.json


tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.metadata:kMDItemWhereFroms'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'


ds-technical-evaluation-data/profile.json


In [0]:
# Criando um Unity Catalog volume (run once)
spark.sql(
    "CREATE VOLUME IF NOT EXISTS workspace.default.ifood_case_volume"
)

volume_path = "/Volumes/workspace/default/ifood_case_volume"
dbutils.fs.mkdirs(volume_path)

True

# 2.0 Organização da Camada Raw de Dados

- Os arquivos de dados no formato JSON foram copiados do DBFS (meu workspace : /Workspace/Users/ruivonatalia9@gmail.com/ifood_case/ds-technical-evaluation-data/) para um Volume governado pelo Unity Catalog, utilizando utilitários nativos do Databricks.
- Em seguida os dados são armazenados em dataframe no catalogo

In [0]:
dbutils.fs.cp(
    "dbfs:/Workspace/Users/ruivonatalia9@gmail.com/ifood_case/ds-technical-evaluation-data/offers.json",
    f"{volume_path}/raw/offers.json"
)

True

In [0]:
dbutils.fs.cp(
    "dbfs:/Workspace/Users/ruivonatalia9@gmail.com/ifood_case/ds-technical-evaluation-data/transactions.json",
    f"{volume_path}/raw/transactions.json"
)

True

In [0]:
dbutils.fs.cp(
    f"dbfs:/Workspace/Users/ruivonatalia9@gmail.com/ifood_case/ds-technical-evaluation-data/profile.json",
    f"{volume_path}/raw/profile.json"
)

True

In [0]:

base_path = "/Volumes/workspace/default/ifood_case_volume/raw/"

transactions_df = spark.read.json(f"{base_path}/transactions.json")
offers_df       = spark.read.json(f"{base_path}/offers.json")
profile_df      = spark.read.json(f"{base_path}/profile.json")


# 3.0 ETL 

- 3.1 Inclusão do campo tempo de registro do cliente no dataframe profile_df
- 3.2 Verificação dos campos offer id x offer_id dentro do dataframe transactions_df
- 3.3 Join entre os dados 
- 3.4 Construção do dataset de features agregado por cliente para construção do modelo 

In [0]:
profile_df = (
    profile_df
    .withColumn(
        "registered_dt",
        to_date(col("registered_on").cast("string"), "yyyyMMdd")
    )
    .withColumn(
        "days_since_registration",
        datediff(date_sub(current_date(), 1), col("registered_dt"))
    )
)

In [0]:
from pyspark.sql.functions import *

# Existem dois campos relacionados à oferta nos eventos:
# - offer_id  (campo aninhado dentro de value)
# - offer id  (campo aninhado dentro de value)
#
# Observação inicial: na maior parte dos casos, quando um está nulo, o outro está preenchido.
# O objetivo desta análise é validar se esses campos se complementam,
# de forma que possamos unificá-los em uma única coluna (offer_id_final),
# maximizando o aproveitamento dos dados disponíveis.

df_confere_offer = (
    transactions_df
    .select("value.*")
    .withColumn(
        "confere_offer",
        when(
            # ambos nulos
            col("offer_id").isNull() & col("offer id").isNull(), True
        ).when(
            # apenas offer_id preenchido
            col("offer_id").isNotNull() & col("offer id").isNull(), True
        ).when(
            # apenas offer id preenchido
            col("offer_id").isNull() & col("offer id").isNotNull(), True
        ).otherwise(False)  # ambos preenchidos (potencial conflito)
    )
)

value_confere_true = (
    df_confere_offer
    .filter(col("confere_offer") == True)
    .agg(count("*"))
    .collect()[0][0]
)

value_total = df_confere_offer.count()

perc_confere_offer = value_confere_true / value_total * 100

print(f"Total de linhas analisadas: {value_total}")
print(f"Linhas em que os campos offer_id e offer id se complementam ou ambos são nulos: {value_confere_true}")
print(f"Percentual de linhas válidas para unificação: {perc_confere_offer}%")

print(
    "Conclusão: os campos offer_id e offer id se comportam de forma complementar "
    "na grande maioria dos casos, o que permite a criação de uma coluna unificada "
    "(offer_id_final) sem perda relevante de informação."
)

transactions_df = (
    transactions_df
    .select("*", "value.*")
    .withColumn(
        "offer_id_final",
        coalesce(col("offer_id"), col("offer id"))
    )
    .drop("value","offer_id", "offer id")
)


Total de linhas analisadas: 306534
Linhas em que os campos offer_id e offer id se complementam ou ambos são nulos: 306534
Percentual de linhas válidas para unificação: 100.0%
Conclusão: os campos offer_id e offer id se comportam de forma complementar na grande maioria dos casos, o que permite a criação de uma coluna unificada (offer_id_final) sem perda relevante de informação.


In [0]:
df_join = (
    transactions_df
    .join( 
        offers_df,
        transactions_df["offer_id_final"]==offers_df["id"], 
        how="left"
    )
    .drop(col('id'))
    .join( 
        profile_df, 
        transactions_df["account_id"]==profile_df["id"], 
        how="left"
        )
    .drop(col('id'))
    
)


## 3.4 Construção do dataset de features (df_agg_geral)

Na célula abaixo é exemplificado todo o pipeline de transformação agregada por account considerando os eventos, canais, conversão e perfil dos clientes.
A geração de um dataset agregado será utilizado como base de features para análises e modelos.
Foi utilizado um display de account de exemplo para exemplificar passo a passo a transformação.

In [0]:

from pyspark.sql.window import *
from pyspark.sql.functions import *

# ------------------------------------------------------------------------------
# 1. Definição da janela temporal dos eventos
# - Particiona por account_id
# - Ordena cronologicamente pelo tempo desde o início do teste
# - Em caso de empate de tempo, aplica uma ordem lógica de eventos
#   (received -> viewed -> completed -> transaction)
# ------------------------------------------------------------------------------
windowSpec = (
    Window
    .partitionBy("account_id")  
    .orderBy(
        col("time_since_test_start").asc(),
        col("event_order").asc()
    )
)

# ------------------------------------------------------------------------------
# 2. Construção do dataframe base (join de eventos, ofertas e perfil)
# ------------------------------------------------------------------------------
df_join = (
    transactions_df
    .join(
        offers_df,
        transactions_df["offer_id_final"] == offers_df["id"],
        how="left"
    )
    .drop(col("id"))
    .join(
        profile_df,
        transactions_df["account_id"] == profile_df["id"],
        how="left"
    )
    .drop(col("id"))
)

# ------------------------------------------------------------------------------
# 3. Enriquecimento temporal e criação de variáveis de histórico
# - event_order: ordem lógica do evento
# - rank_events: ranking temporal por conta
# - prev_reward: reward do evento anterior
# - prev_offer_type: tipo da oferta anterior associada à conversão
# ------------------------------------------------------------------------------
df_join = (
    df_join
    .withColumn(
        "event_order",
        when(col("event") == "offer received", 1)
        .when(col("event") == "offer viewed", 2)
        .when(col("event") == "offer completed", 3)
        .otherwise(4)
    )
    .withColumn("rank_events", rank().over(windowSpec))
    .withColumn("prev_reward", lag("reward").over(windowSpec))
    .withColumn("prev_offer_type", lag("offer_type").over(windowSpec))
    .withColumn(
        "prev_offer_type",
        when(col("prev_reward").isNotNull(), col("prev_offer_type"))
    )
    .withColumn("rewardd", col("reward"))
    .orderBy(col("time_since_test_start"), col("event_order"))
)

# ------------------------------------------------------------------------------
# 4. Validação visual – linha do tempo completa de uma conta exemplo
# ------------------------------------------------------------------------------
print("▶️ Linha do tempo completa de eventos – account de teste")

display(
    df_join
    .filter(col("account_id") == "9fa9ae8f57894cc9a3b8a9bbe0fc1b2f")
)

# ------------------------------------------------------------------------------
# 5. Filtro de eventos relevantes para conversão
# Mantém apenas eventos que possuem reward direto ou reward anterior
# ------------------------------------------------------------------------------
print(" Eventos associados a conversão – account de teste")

display(
    df_join
    .filter(col("account_id") == "9fa9ae8f57894cc9a3b8a9bbe0fc1b2f")
    .filter(col("prev_reward").isNotNull() | col("reward").isNotNull())
)

# ------------------------------------------------------------------------------
# 6. Agregações de conversão por tipo de oferta
# ------------------------------------------------------------------------------
df_agg_conversao = (
    df_join
    .filter(col("prev_reward").isNotNull() | col("reward").isNotNull())
    .groupBy("account_id")
    .agg(
        sum(when(col("offer_type") == "bogo", 1).otherwise(0)).alias("bogo_conversao"),
        sum(when(col("offer_type") == "discount", 1).otherwise(0)).alias("discount_conversao"),
        sum(when(col("offer_type") == "informational", 1).otherwise(0)).alias("informational_conversao"),
        avg("discount_value").alias("avg_discount_value_conversao"),
        avg("duration").alias("avg_duration_conversao"),
        avg("min_value").alias("avg_min_value_conversao"),
        sum("reward").alias("reward_conversao"),
        sum("amount").alias("amount_conversao"),
        count("prev_reward").alias("qt_transacional_conversao")
    )
)

print("▶️ Métricas agregadas de conversão – account de teste")

display(
    df_agg_conversao
    .filter(col("account_id") == "9fa9ae8f57894cc9a3b8a9bbe0fc1b2f")
)

# ------------------------------------------------------------------------------
# 7. Janela temporal das conversões
# ------------------------------------------------------------------------------
df_agg_conversao_min_time_test = (
    df_join
    .filter(col("prev_reward").isNotNull() | col("reward").isNotNull())
    .filter(col("event") == "transaction")
    .groupBy("account_id")
    .agg(
        min("time_since_test_start").alias("min_time_conversao"),
        max("time_since_test_start").alias("max_time_conversao")
    )
)

print(" Janela temporal das conversões – account de teste")

display(
    df_agg_conversao_min_time_test
    .filter(col("account_id") == "9fa9ae8f57894cc9a3b8a9bbe0fc1b2f")
)

# ------------------------------------------------------------------------------
# 8. Frequência de canais associados à conversão
# ------------------------------------------------------------------------------
df_agg_channels = (
    df_join
    .filter(col("prev_reward").isNotNull() | col("reward").isNotNull())
    .groupBy("account_id")
    .agg(
        expr("""
            transform(
                array_distinct(flatten(collect_list(channels))),
                c -> struct(
                    c as channel,
                    size(filter(flatten(collect_list(channels)), x -> x = c)) as frequency
                )
            )
        """).alias("channels_frequency")
    )
    .withColumn("channel_web_frequency", expr("aggregate(filter(channels_frequency, x -> x.channel = 'web'), 0, (acc, x) -> acc + x.frequency)"))
    .withColumn("channel_email_frequency", expr("aggregate(filter(channels_frequency, x -> x.channel = 'email'), 0, (acc, x) -> acc + x.frequency)"))
    .withColumn("channel_mobile_frequency", expr("aggregate(filter(channels_frequency, x -> x.channel = 'mobile'), 0, (acc, x) -> acc + x.frequency)"))
    .withColumn("channel_social_frequency", expr("aggregate(filter(channels_frequency, x -> x.channel = 'social'), 0, (acc, x) -> acc + x.frequency)"))
)

print(" Frequência de canais de conversão – account de teste")

display(
    df_agg_channels
    .filter(col("account_id") == "9fa9ae8f57894cc9a3b8a9bbe0fc1b2f")
)

# ------------------------------------------------------------------------------
# 9. Agregação de eventos (funil)
# ------------------------------------------------------------------------------
df_agg_event = (
    df_join
    .groupBy("account_id")
    .pivot("event")
    .count()
)

df_agg_event = df_agg_event.select(
    [
        col(c).alias(
            c.replace(" ", "_")
             .replace("offer", "qt_offer")
             .replace("transaction", "qt_transaction")
        )
        for c in df_agg_event.columns
    ]
)

print("Contagem de eventos por tipo – account de teste")

display(
    df_agg_event
    .filter(col("account_id") == "9fa9ae8f57894cc9a3b8a9bbe0fc1b2f")
)

# ------------------------------------------------------------------------------
# 10. Dataset final de features por conta
# ------------------------------------------------------------------------------
df_agg_geral = (
    df_join
    .groupBy("account_id", "age", "gender")
    .agg(
        sum("amount").alias("gmv_total"),
        avg("days_since_registration").alias("days_since_registration"),
        max("credit_card_limit").alias("max_credit_card_limit"),
        sum("prev_reward").alias("tot_subsidio_utilizado")
    )
    .join(df_agg_event, on="account_id", how="left")
    .join(df_agg_channels, on="account_id", how="left")
    .join(df_agg_conversao, on="account_id", how="left")
    .join(df_agg_conversao_min_time_test, on="account_id", how="left")
)

df_agg_geral = df_agg_geral.select(
    [col(c).alias(c.replace("frequency", "frequency_conversao")) for c in df_agg_geral.columns]
)

print("Dataset final de features – account de teste")

display(
    df_agg_geral
    .filter(col("account_id") == "9fa9ae8f57894cc9a3b8a9bbe0fc1b2f")
)


▶️ Linha do tempo completa de eventos – account de teste


account_id,event,time_since_test_start,amount,reward,offer_id_final,channels,discount_value,duration,min_value,offer_type,age,credit_card_limit,gender,registered_on,registered_dt,days_since_registration,event_order,rank_events,prev_reward,prev_offer_type,rewardd
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer received,0.0,null,null,2906b810c7d4411798c6938adc9daaa5,"List(web, email, mobile)",2,7.0,10,discount,42,96000.0,M,20160117,2016-01-17,3645,1,1,null,null,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer viewed,0.0,null,null,2906b810c7d4411798c6938adc9daaa5,"List(web, email, mobile)",2,7.0,10,discount,42,96000.0,M,20160117,2016-01-17,3645,2,2,null,null,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0.0,null,2.0,2906b810c7d4411798c6938adc9daaa5,"List(web, email, mobile)",2,7.0,10,discount,42,96000.0,M,20160117,2016-01-17,3645,3,3,null,null,2.0
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0.0,34.56,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,4,2.0,discount,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,1.75,21.55,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,5,null,null,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,4.75,32.87,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,6,null,null,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,6.5,19.99,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,7,null,null,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,7.0,20.97,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,8,null,null,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,8.5,31.09,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,9,null,null,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer received,14.0,null,null,2298d6c36e964ae4a3e7e9706d1fb8c2,"List(web, email, mobile, social)",3,7.0,7,discount,42,96000.0,M,20160117,2016-01-17,3645,1,10,null,null,null


 Eventos associados a conversão – account de teste


account_id,event,time_since_test_start,amount,reward,offer_id_final,channels,discount_value,duration,min_value,offer_type,age,credit_card_limit,gender,registered_on,registered_dt,days_since_registration,event_order,rank_events,prev_reward,prev_offer_type,rewardd
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,0.0,null,2.0,2906b810c7d4411798c6938adc9daaa5,"List(web, email, mobile)",2,7.0,10,discount,42,96000.0,M,20160117,2016-01-17,3645,3,3,null,null,2.0
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,0.0,34.56,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,4,2.0,discount,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,14.75,null,3.0,2298d6c36e964ae4a3e7e9706d1fb8c2,"List(web, email, mobile, social)",3,7.0,7,discount,42,96000.0,M,20160117,2016-01-17,3645,3,12,null,null,3.0
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,14.75,18.42,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,13,3.0,discount,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,19.75,null,5.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,"List(web, email)",5,10.0,20,discount,42,96000.0,M,20160117,2016-01-17,3645,3,19,null,null,5.0
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,19.75,21.13,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,20,5.0,discount,null
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,offer completed,22.5,null,10.0,4d5c57ea9a6940dd891ad53e9dbe8da0,"List(web, email, mobile, social)",10,5.0,10,bogo,42,96000.0,M,20160117,2016-01-17,3645,3,23,null,null,10.0
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,transaction,22.5,24.3,null,null,null,null,null,null,null,42,96000.0,M,20160117,2016-01-17,3645,4,24,10.0,bogo,null


▶️ Métricas agregadas de conversão – account de teste


...

account_id,channels_frequency,channel_web_frequency,channel_email_frequency,channel_mobile_frequency,channel_social_frequency
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,"List(List(web, 4), List(email, 4), List(mobile, 3), List(social, 2))",4,4,3,2


Contagem de eventos por tipo – account de teste


account_id,qt_offer_completed,qt_offer_received,qt_offer_viewed,qt_transaction
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,4,4,4,12


Dataset final de features – account de teste


account_id,age,gender,gmv_total,days_since_registration,max_credit_card_limit,tot_subsidio_utilizado,qt_offer_completed,qt_offer_received,qt_offer_viewed,qt_transaction,channels_frequency_conversao,channel_web_frequency_conversao,channel_email_frequency_conversao,channel_mobile_frequency_conversao,channel_social_frequency_conversao,bogo_conversao,discount_conversao,informational_conversao,avg_discount_value_conversao,avg_duration_conversao,avg_min_value_conversao,reward_conversao,amount_conversao,qt_transacional_conversao,min_time_conversao,max_time_conversao
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,42,M,320.48,3645.0,96000.0,20.0,4,4,4,12,"List(List(web, 4), List(email, 4), List(mobile, 3), List(social, 2))",4,4,3,2,1,3,0,5.0,7.25,11.75,20.0,98.41,4,0.0,22.5


# 4.0 Escrita do dataframe

In [0]:
df_agg_geral.write.mode("overwrite").saveAsTable("dataset_features")


In [0]:
display(
    df_agg_geral
    .limit(10)
    )

account_id,age,gender,gmv_total,days_since_registration,max_credit_card_limit,tot_subsidio_utilizado,qt_offer_completed,qt_offer_received,qt_offer_viewed,qt_transaction,channels_frequency_conversao,channel_web_frequency_conversao,channel_email_frequency_conversao,channel_mobile_frequency_conversao,channel_social_frequency_conversao,bogo_conversao,discount_conversao,informational_conversao,avg_discount_value_conversao,avg_duration_conversao,avg_min_value_conversao,reward_conversao,amount_conversao,qt_transacional_conversao,min_time_conversao,max_time_conversao
03833a1fe27f4435917da7e54c6fa48a,118,null,2.38,2849.0,null,null,null,4,4,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
dd69125d21a54693840469a1ad6ca243,66,M,15.99,3006.0,32000.0,3.0,1,3,3,6,"List(List(web, 1), List(email, 1), List(mobile, 1), List(social, 1))",1,1,1,1,0,1,0,3.0,7.0,7.0,3.0,4.2,1,4.0,4.0
3d55f046a320408db9a88be59acc19cc,118,null,24.46,3432.0,null,2.0,1,6,4,12,"List(List(web, 1), List(email, 1), List(mobile, 1), List(social, 1))",1,1,1,1,0,1,0,2.0,10.0,10.0,2.0,2.59,1,25.0,25.0
752045b74388467d99a44eac9501116e,43,F,14.229999999999999,3045.0,42000.0,null,null,5,2,5,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
b66cb315b9944a749e25aee0d7dfe13c,87,M,84.94999999999999,2903.0,33000.0,7.0,3,6,5,9,"List(List(web, 3), List(email, 3), List(mobile, 3), List(social, 2))",3,3,3,2,0,3,0,2.3333333333333335,8.0,9.0,7.0,24.36,3,17.25,28.75
04e5255c9e754c66b83c06ec70f62203,93,M,93.56,3049.0,35000.0,21.0,6,6,6,11,"List(List(web, 6), List(email, 6), List(mobile, 5), List(social, 4))",6,6,5,4,2,4,0,3.5,7.833333333333333,10.0,21.0,47.59,6,5.25,25.0
614ec3d641654c2d9d5c2a15e329c774,67,F,239.51000000000002,3217.0,96000.0,23.0,5,5,5,9,"List(List(web, 4), List(email, 5), List(mobile, 5), List(social, 3))",4,5,5,3,2,3,0,4.6,7.0,7.8,23.0,130.59,5,2.0,21.0
6924192984cf4a17ac5e5b1a937af4b2,32,M,137.46,2808.0,53000.0,2.0,1,4,4,7,"List(List(web, 1), List(email, 1), List(mobile, 1), List(social, 1))",1,1,1,1,0,1,0,2.0,10.0,10.0,2.0,2.27,1,9.5,9.5
b4d5401786f444169c723533ed475760,23,F,22.39,2796.0,43000.0,5.0,1,6,3,9,"List(List(web, 1), List(email, 1), List(mobile, 1), List(social, 1))",1,1,1,1,1,0,0,5.0,5.0,5.0,5.0,6.98,1,26.0,26.0
b797b614e6514d55af41629b9d3f2ba2,71,F,130.65,3475.0,34000.0,9.0,3,5,4,17,"List(List(web, 3), List(email, 3), List(mobile, 2), List(social, 1))",3,3,2,1,0,3,0,3.0,9.0,13.333333333333334,9.0,27.25,3,2.25,27.0
